In [34]:
%matplotlib inline  

import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import sqlite3
import random
import tables
import pickle
from scipy.spatial.distance import cdist
from collections import Counter
import graphlab as gl
from sklearn import preprocessing
from sklearn.decomposition import PCA

from IPython.html.widgets import FloatProgress
from IPython.core.display import HTML
from IPython.display import display

In [27]:
data = np.load("/home/kayibal/sc-recom/code/msd_data/final_features.npy").T

In [653]:
with open("/home/kayibal/sc-recom/code/msd_data/eval_dic.p", "r") as f:
    splitted_user_likes = pickle.load(f)

In [627]:
with open("/home/kayibal/sc-recom/code/msd_data/eval_dic_big.p", "r") as f:
    splitted_likes = pickle.load(f)

In [7]:
with open("/home/kayibal/sc-recom/code/msd_data/like_dic.p", "r") as f:
    user_likes = pickle.load(f)

In [626]:
with open("/home/kayibal/sc-recom/code/msd_data/like_dic_big.p", "r") as f:
    user_likes = pickle.load(f)

In [247]:
#load feature vectos
f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis2.h5", mode="r")

minmax = preprocessing.MinMaxScaler()

pca = PCA(n_components=60)
flucs_comp = pca.fit_transform(np.asarray(minmax.fit_transform(f.root.analysis.flucs)).transpose()).transpose()

data = {'A':{},'B':{},'C':{},'D':{},'E':{},'F':{},'G':{},'H':{},'I':{},'J':{}}
data['A']['desc'] = "Fluctuation Patterns + Gaussian Concatenated MFCC Representations"
data['A']['data'] = np.vstack((np.array(flucs_comp), minmax.fit_transform(np.array(f.root.analysis.gaussian_rep)))).transpose()
data['B']['desc'] = "Fluctuation Patterns + Concatenated MFCC Medians"
data['B']['data'] = np.vstack((np.array(flucs_comp), minmax.fit_transform(np.array(f.root.analysis.mfcc_medians_comp)))).transpose()
data['C']['desc'] = "Fluctuation Patterns + Concatenated MFCC Delta Medians"
data['C']['data'] = np.vstack((np.array(flucs_comp), minmax.fit_transform(np.array(f.root.analysis.mfcc_delta_medians_comp)))).transpose()
data['D']['desc'] = "Fluctuation Patterns + Concatenated MFCC Medians and Delta Medians"
data['D']['data'] = np.vstack((np.array(flucs_comp), minmax.fit_transform(np.array(f.root.analysis.mfcc_medians_comp)), minmax.fit_transform(np.array(f.root.analysis.mfcc_delta_medians_comp)))).transpose()
data['E']['desc'] = "Fluctuation Patterns Compressed"
data['E']['data'] = np.array(flucs_comp).transpose()
data['F']['desc'] = "Concatenated MFCC Medians"
data['F']['data'] = minmax.fit_transform(np.array(f.root.analysis.mfcc_medians_comp).transpose())
data['G']['desc'] = "Concatenated MFCC Delta Medians"
data['G']['data'] = minmax.fit_transform(np.array(f.root.analysis.mfcc_delta_medians_comp).transpose())
data['H']['desc'] = "Concatenated MFCC Medians and Delta Medians"
data['H']['data'] = np.vstack((minmax.fit_transform(np.array(f.root.analysis.mfcc_medians_comp)), minmax.fit_transform(np.array(f.root.analysis.mfcc_delta_medians_comp)))).transpose()
data['I']['desc'] = "Gaussian Concatenated MFCC Representations"
data['I']['data'] = minmax.fit_transform(np.array(f.root.analysis.gaussian_rep)).transpose()
data['J']['desc'] = "Raw Fluctuation Patterns"
data['J']['data'] = np.array(f.root.analysis.flucs).transpose()
f.close()

In [12]:
#make sframe from user likes
likes = {'user_id':[], 'item_id':[]}
for user in splitted_likes.keys():
    for track in splitted_likes[user]['rec']:
        likes['user_id'].append(user)
        likes['item_id'].append(track)
print len(Counter(likes['user_id']))
sf = gl.SFrame(likes)
cf = gl.recommender.create(sf)

250
PROGRESS: Recsys training: model = item_similarity
PROGRESS: Preparing data set.
PROGRESS:     Data has 11925 observations with 250 users and 7366 items.
PROGRESS:     Data prepared in: 0.040732s
PROGRESS: Computing item similarity statistics:
PROGRESS: Computing most similar items for 7366 items:
PROGRESS: +-----------------+-----------------+
PROGRESS: | Number of items | Elapsed Time    |
PROGRESS: +-----------------+-----------------+
PROGRESS: | 1000            | 1.60743         |
PROGRESS: | 2000            | 1.66643         |
PROGRESS: | 3000            | 1.71731         |
PROGRESS: | 4000            | 1.75434         |
PROGRESS: | 5000            | 1.7868          |
PROGRESS: | 6000            | 1.93495         |
PROGRESS: | 7000            | 2.07185         |
PROGRESS: +-----------------+-----------------+
PROGRESS: Finished training in 2.28384s


In [630]:
popcf = gl.popularity_recommender.create(sf)

PROGRESS: Recsys training: model = popularity
PROGRESS: Preparing data set.
PROGRESS:     Data has 651754 observations with 16817 users and 7979 items.
PROGRESS:     Data prepared in: 1.04798s
PROGRESS: 651754 observations to process; with 7979 unique items.


In [13]:
#database
conn = sqlite3.connect("/home/kayibal/sc-recom/code/msd_data/analysis_data/sampled_likes_tracks.db")
c = conn.cursor()
c.execute("SELECT msd_id FROM tracks WHERE audio_path IS NOT NULL ORDER BY msd_id")
tracks = c.fetchall()
conn.close()
tracks = np.array([el[0] for el in tracks])

In [14]:
def retrieve_liked_clusters(cluster,likes):
    liked_clusters = []
    query = "SELECT %s_cluster FROM tracks WHERE msd_id = '%s'"
    
    conn = sqlite3.connect("/home/kayibal/sc-recom/code/msd_data/analysis_data/sampled_likes_tracks.db")
    c = conn.cursor()
    for msd_id in likes:
        c.execute(query % (cluster,msd_id))
        liked_clusters.append(c.fetchone()[0])
    conn.close()
    return liked_clusters

In [15]:
def make_recommendations(cluster, likes, remove_likes=True):
    liked_clusters = set(retrieve_liked_clusters(cluster,likes))
    #print "this user liked %d of 400 clusters" % len(liked_clusters)
    rec = []
    query = "SELECT msd_id FROM tracks WHERE %s_cluster = '%d'"
    conn = sqlite3.connect("/home/kayibal/sc-recom/code/msd_data/analysis_data/sampled_likes_tracks.db")
    c = conn.cursor()
    for c_id in set(liked_clusters):
        c.execute(query%(cluster,c_id))
        rec += [r[0] for r in c.fetchall()]
    if remove_likes:
        itemsToRemove = set(likes)
        recs = filter(lambda x: x not in itemsToRemove,recs)
    return rec

In [16]:
def make_recommendations_freq_cluster(cluster, likes, cutoff, remove_likes=True):
    liked_clusters = [el[0] for el in Counter(retrieve_liked_clusters(cluster,likes)).most_common()[:cutoff]]
    #print "this user liked %d of 400 clusters" % len(liked_clusters)
    rec = []
    query = "SELECT msd_id FROM tracks WHERE %s_cluster = '%d'"
    conn = sqlite3.connect("/home/kayibal/sc-recom/code/msd_data/analysis_data/sampled_likes_tracks.db")
    c = conn.cursor()
    for c_id in set(liked_clusters):
        c.execute(query%(cluster,c_id))
        rec += [r[0] for r in c.fetchall()]
    conn.close()
    if remove_likes:
        itemsToRemove = set(likes)
        rec = filter(lambda x: x not in itemsToRemove,rec)
    return rec

In [17]:
def get_features(msd_ids, data):
    idx_arr = []
    ids = set(msd_ids)
    for i,t in enumerate(tracks):
        if t in ids:
            idx_arr.append(i)
    return data[:,idx_arr]

In [18]:
def make_recommendations_scored(cluster, likes, cutoff, data, remove_likes=True):
    liked_clusters = [el[0] for el in Counter(retrieve_liked_clusters(cluster,likes)).most_common()]
    #print "this user liked %d of 400 clusters" % len(liked_clusters)
    rec = []
    query = "SELECT msd_id FROM tracks WHERE %s_cluster = '%d'"
    conn = sqlite3.connect("/home/kayibal/sc-recom/code/msd_data/analysis_data/sampled_likes_tracks.db")
    c = conn.cursor()
    for c_id in set(liked_clusters):
        c.execute(query%(cluster,c_id))
        rec += [r[0] for r in c.fetchall()]
    conn.close()
    rec.sort()
    #calculate distance matrix between likes and recs
    like_features = get_features(likes,data)
    rec_features = get_features(rec,data)
    scores = np.sum(cdist(like_features.T,rec_features.T,'euclidean'), axis=0)
    scored_recs = sorted(zip(rec,scores), key= lambda x: x[1])
    recs = [el[0] for el in scored_recs]
    if remove_likes:
        itemsToRemove = set(likes)
        recs = filter(lambda x: x not in itemsToRemove,recs)
    return recs[:cutoff]

In [346]:
def make_recommendations_cluster_scored(cluster, likes, cutoff, data, k=3, remove_likes=True, score="distance"):
    liked_clusters = [el[0] for el in Counter(retrieve_liked_clusters(cluster,likes)).most_common()][:k]
    like_set = set(likes)
    recs = []
    query = "SELECT msd_id FROM tracks WHERE %s_cluster = '%d' ORDER BY msd_id"
    conn = sqlite3.connect("/home/kayibal/sc-recom/code/msd_data/analysis_data/sampled_likes_tracks.db")
    c = conn.cursor()
    for c_id in set(liked_clusters):
        c.execute(query%(cluster, c_id))
        temp = [x[0] for x in c.fetchall()]
        if score == "distance":
            #likes which belong to c_id
            c_likes = set(temp) & like_set
            temp = filter(lambda x: x not in like_set,temp)
            #calculate score and normalize
            like_features = get_features(c_likes,data)
            rec_features = get_features(temp,data)
            try:
                scores = np.min(cdist(like_features.T,rec_features.T,'euclidean'), axis=0)
                scores = scores / np.max(scores)
                scored_recs = sorted(zip(temp,scores), key= lambda x: x[1])
                recs += scored_recs
            except ValueError:
                continue
        else:
            recs += list(popcf.recommend(items=temp)['item_id'])[:(cutoff/k)]
    conn.close()
    #calculate distance matrix between likes and recs
    if score == "distance":
        recs = sorted(recs, key= lambda x: x[1])
        recs = [el[0] for el in recs]
    if remove_likes:
        recs = filter(lambda x: x not in like_set,recs)
    return recs[:cutoff]

In [266]:
def dist_matrix_recommender(likes, feature, cutoff):
    recs=[]
    dists = []
    for like in likes:
        like_idx = np.where(tracks == like)[0][0]
        feature_vec = data[feature]['data'][like_idx:like_idx+1,:]
        distances = cdist(feature_vec, data[feature]['data']).flatten()
        indizes = np.argsort(distances)[1:cutoff/len(likes)]
        dists += list(distances[1:cutoff/len(likes)])
        recs += list(tracks[indizes])
    scored_recs = zip(recs,dists)
    scored_recs = sorted(scored_recs, key = lambda x:x[1])
    return [el[0] for el in scored_recs][:cutoff]

In [20]:
def make_cf_recommendations(likes, cutoff,remove_likes=True):
    recs = cf.get_similar_items(items = likes).sort('score', ascending=False)['similar']
    if remove_likes:
        itemsToRemove = set(likes)
        recs = filter(lambda x: x not in itemsToRemove,recs)
    return recs[:cutoff]

In [21]:
def make_cf_user_recommendations(user, cutoff,remove_likes=True):
    recs = cf.recommend([user],k=4000).sort('score', ascending=False)['item_id']
    if remove_likes:
        itemsToRemove = set(likes)
        recs = filter(lambda x: x not in itemsToRemove,recs)
    return recs[:cutoff]

In [22]:
def make_random_recommendations(length,remove_likes=True):
    recs = random.sample(tracks, length+len(likes))
    if remove_likes:
        itemsToRemove = set(likes)
        recs = filter(lambda x: x not in itemsToRemove,recs)
    return recs[:length]

In [23]:
def display_scores(scores):
    row_html = "<tr><td>%d</td><td>%.2f%% (&#177; %.2f)</td><td>%.2f%% (&#177; %.2f)</td><td>%.2f%% (&#177; %.2f)</td><td>%.2f%% (&#177; %.2f)</td></tr>"
    first_row = "<tr><td rowspan='%d'>%s</td><td>%d</td><td>%.2f%% (&#177; %.2f)</td><td>%.2f%% (&#177; %.2f)</td><td>%.2f%% (&#177; %.2f)</td><td>%.2f%% (&#177; %.2f)</td></tr>"
    html = "<table style='width:100%'>"
    #headers
    html += "<tr><th>%s</th><th>%s</th><th>%s</th><th>%s</th><th>%s</th><th>%s</th></tr>" % ("Feature</br> Vector","Clusters used","Our Approach", "CF Recommender","Random Recommender", "Recommendation Size (% of total songs)")
    for key in scores.keys():
        first = True
        for row in scores[key]:
            if first:
                html += first_row % (len(scores[key]), key, row['cutoff'], row['tp_mean'], row['tp_std'], row['tp_cf_mean'], row['tp_cf_std'], row['tp_rand_mean'], row['tp_rand_std'], row['per_mean'], row['per_std'])
                first = False
            else:
                html += row_html % (row['cutoff'], row['tp_mean'], row['tp_std'], row['tp_cf_mean'], row['tp_cf_std'], row['tp_rand_mean'], row['tp_rand_std'], row['per_mean'], row['per_std'])
    html += "</table>"
    display(HTML(html))

In [24]:
def scores_latex(scores):
    
    row_string = "& $%d$ & $%.2f\\%% (\\pm %.2f) $&$ %.2f\\%% (\\pm %.2f) $&$ %.2f\\%% (\\pm %.2f) $&$ %.2f\\%% (\\pm %.2f) $ \\\\ \\cline{2-6} \n"
    first_row = "\\hline \\hline \\multirow{%d}{*}{%s} & $%d$ &$ %.2f\\%% (\\pm %.2f) $&$ %.2f\\%% (\\pm %.2f) $&$ %.2f\\%% (\\pm %.2f) $&$ %.2f\\%% (\\pm %.2f)$ \\\\ \\cline{2-6} \n"
    html = "\\begin{tabular}{| l || c | c | c | c | c |}\n \\hline \n"
    #headers
    html += "\\rowcolor{lightgray} & %s & %s & %s & %s & %s \\\\ \\hline \n" % ("Clusters used","Our Approach", "CF Recommender","Random Recommender", "Recommendation Size (\\% of total songs)")
    for key in scores.keys():
        first = True
        for row in scores[key]:
            if first:
                html += first_row % (len(scores[key]), key, row['cutoff'], row['tp_mean'], row['tp_std'], row['tp_cf_mean'], row['tp_cf_std'], row['tp_rand_mean'], row['tp_rand_std'], row['per_mean'], row['per_std'])
                first = False
            else:
                html += row_string % (row['cutoff'], row['tp_mean'], row['tp_std'], row['tp_cf_mean'], row['tp_cf_std'], row['tp_rand_mean'], row['tp_rand_std'], row['per_mean'], row['per_std'])
    html += "\\end{tabular}\n"
    return html

In [287]:
print scores_latex(all_scores)

\begin{tabular}{| l || c | c | c | c | c |}
 \hline 
\rowcolor{lightgray} & Clusters used & Our Approach & CF Recommender & Random Recommender & Recommendation Size (\% of total songs) \\ \hline 
\hline \hline \multirow{4}{*}{A} & $1$ &$ 0.87\% (\pm 4.37) $&$ 0.84\% (\pm 4.34) $&$ 0.47\% (\pm 3.62) $&$ 0.62\% (\pm 0.00)$ \\ \cline{2-6} 
& $5$ & $3.86\% (\pm 9.36) $&$ 3.21\% (\pm 8.88) $&$ 3.28\% (\pm 8.93) $&$ 3.12\% (\pm 0.00) $ \\ \cline{2-6} 
& $20$ & $13.89\% (\pm 17.77) $&$ 9.73\% (\pm 13.62) $&$ 12.02\% (\pm 13.96) $&$ 12.50\% (\pm 0.00) $ \\ \cline{2-6} 
& $40$ & $27.49\% (\pm 21.36) $&$ 18.26\% (\pm 19.34) $&$ 25.40\% (\pm 20.99) $&$ 24.99\% (\pm 0.00) $ \\ \cline{2-6} 
\hline \hline \multirow{4}{*}{C} & $1$ &$ 0.62\% (\pm 4.19) $&$ 0.84\% (\pm 4.34) $&$ 0.58\% (\pm 3.40) $&$ 0.62\% (\pm 0.00)$ \\ \cline{2-6} 
& $5$ & $3.72\% (\pm 9.05) $&$ 3.21\% (\pm 8.88) $&$ 2.90\% (\pm 7.79) $&$ 3.12\% (\pm 0.00) $ \\ \cline{2-6} 
& $20$ & $13.12\% (\pm 16.16) $&$ 9.73\% (\pm 13.62) $&$ 12

In [275]:
def get_scores(cluster, like_dict, cutoffs):
    f = FloatProgress(min=0, max=100)
    display(f)
    scores = []
    for c in cutoffs:
        user_scores = []
        idx = 0
        for user in like_dict.keys():
            f.value = idx*100./len(like_dict.keys())
            f.description = "current cutoff: %d" % c
            idx += 1
            #recs = make_recommendations_freq_cluster(cluster, like_dict[user]['rec'],c)
            recs = make_recommendations_cluster_scored(cluster, like_dict[user]['rec'],c*50,data[cluster]['data'].T,k=c)
            #recs = dist_matrix_recommender(like_dict[user]['rec'], cluster, c)
            rand_recs = make_random_recommendations(len(recs))
            cf_recs = make_cf_user_recommendations(user, len(recs))
            tp = 0
            tp_cf = 0
            tp_r = 0
            percentage = len(recs)*100./8003.
            for track in like_dict[user]['eval']:
                if track in recs:
                    tp += 1
                if track in rand_recs:
                    tp_r += 1
                if track in cf_recs:
                    tp_cf += 1
            score = tp*100./len(like_dict[user]['eval'])
            score_cf = tp_cf*100./len(like_dict[user]['eval'])            
            score_random = tp_r*100./len(like_dict[user]['eval'])
            user_scores.append([score, score_cf, score_random, percentage])
        mean = np.mean(user_scores, axis=0)
        std = np.std(user_scores, axis=0)
        scores.append({'cutoff':c,'tp_mean':mean[0], 'tp_std':std[0], 'tp_cf_mean':mean[1], 'tp_cf_std':std[1], 'tp_rand_mean':mean[2], 'tp_rand_std':std[2], 'per_mean':mean[3], 'per_std':std[3]})
    return scores

In [336]:
all_scores_mod = {}
for key in 'ABCDEFGHIJ':
    print key
    all_scores_mod[key] = get_scores(key, splitted_likes, [1,5,20,40])
    display_scores(all_scores)

A


KeyboardInterrupt: 

In [335]:
display_scores(all_scores)

In [174]:
make_recommendations_cluster_scored('K', ['TRAGYLG128F14928B5'],3,data,k=10)

[u'TRSTVVF128F42BA445', u'TRDREXV128F146265C', u'TRTSIXM12903CABAA0']

In [244]:
def get_scored_scores(like_dict, rec_lengths=[1800,1000,500,100,20,5]):
    scores = {}
    for l in rec_lengths:
        scores[l]=[]
    for i,user in enumerate(like_dict.keys()):
            print i
            recs = make_recommendations_cluster_scored('H', like_dict[user]['rec'],1800,data, k=40)
            rand_recs = make_random_recommendations(len(recs))
            cf_recs = make_cf_recommendations(like_dict[user]['rec'], len(recs))
            
            for l in rec_lengths:
                tp = 0
                tp_cf = 0
                tp_r = 0
                percentage = len(recs)*100./8003.
                for track in like_dict[user]['eval']:
                    if track in recs[:l]:
                        tp += 1
                    if track in rand_recs[:l]:
                        tp_r += 1
                    if track in cf_recs[:l]:
                        tp_cf += 1
                score = tp*100./len(like_dict[user]['eval'])
                score_cf = tp_cf*100./len(like_dict[user]['eval'])            
                score_random = tp_r*100./len(like_dict[user]['eval'])
                scores[l].append([score, score_cf, score_random, percentage])
    return scores

In [245]:
scored_scores = get_scored_scores(splitted_likes) 

0
PROGRESS: Getting similar items completed in 0.003273
1
PROGRESS: Getting similar items completed in 0.002488
2
PROGRESS: Getting similar items completed in 0.003443
3
PROGRESS: Getting similar items completed in 0.00215
4
PROGRESS: Getting similar items completed in 0.002535
5
PROGRESS: Getting similar items completed in 0.003094
6
PROGRESS: Getting similar items completed in 0.002359
7
PROGRESS: Getting similar items completed in 0.002144
8
PROGRESS: Getting similar items completed in 0.001965
9
PROGRESS: Getting similar items completed in 0.002113
10
PROGRESS: Getting similar items completed in 0.00252
11
PROGRESS: Getting similar items completed in 0.002303
12
PROGRESS: Getting similar items completed in 0.003721
13
PROGRESS: Getting similar items completed in 0.002104
14
PROGRESS: Getting similar items completed in 0.001902
15
PROGRESS: Getting similar items completed in 0.001695
16
PROGRESS: Getting similar items completed in 0.00187
17
PROGRESS: Getting similar items completed

In [246]:
for length,user_scores in scored_scores.iteritems():
    mean =  np.mean(user_scores, axis=0)
    std = np.std(user_scores, axis=0)
    print "$%d$ & $%.2f\\%% (\\pm %.2f) $&$ %.2f\\%% (\\pm %.2f) $&$ %.2f\\%% (\\pm %.2f) $ \\\\ \\hline \n" % (length,mean[0], std[0], mean[1], std[1], mean[2], std[2],) 

$100$ & $1.23\% (\pm 4.62) $&$ 1.05\% (\pm 4.72) $&$ 1.71\% (\pm 6.44) $ \\ \hline 

$5$ & $0.00\% (\pm 0.00) $&$ 0.16\% (\pm 2.14) $&$ 0.00\% (\pm 0.00) $ \\ \hline 

$1800$ & $26.59\% (\pm 23.35) $&$ 2.92\% (\pm 7.02) $&$ 20.56\% (\pm 17.86) $ \\ \hline 

$1000$ & $14.90\% (\pm 18.34) $&$ 2.92\% (\pm 7.02) $&$ 11.00\% (\pm 13.90) $ \\ \hline 

$500$ & $7.37\% (\pm 14.16) $&$ 2.92\% (\pm 7.02) $&$ 5.63\% (\pm 10.15) $ \\ \hline 

$20$ & $0.10\% (\pm 1.58) $&$ 0.36\% (\pm 2.75) $&$ 0.24\% (\pm 1.88) $ \\ \hline 



In [291]:
def count_outperforms(like_dict, feature, rec_lengths=[50,250,1000,2000]):
    A = {}
    B = {}
    f = FloatProgress(min=0, max=100)
    display(f)
    f.description = feature
    for l in rec_lengths:
        A[l] = 0
        B[l] = 0
    for i,user in enumerate(like_dict.keys()):
            f.value = float(i)*100/len(like_dict)
            #recs = make_recommendations_cluster_scored('H', like_dict[user]['rec'],1800, data)
            recs = make_recommendations_cluster_scored(feature, like_dict[user]['rec'],2000,data[feature]['data'].T,k=40)
            #cf_recs = make_cf_recommendations(like_dict[user]['rec'], 1000)
            recs_b = make_random_recommendations(len(recs))
            
            for l in rec_lengths:
                tp = 0
                tp_b = 0
                for track in like_dict[user]['eval']:
                    if track in recs[:l]:
                        tp += 1
                    if track in recs_b[:l]:
                        tp_b += 1
                if(tp > tp_b):
                    A[l] += 1
                elif(tp < tp_b):
                    B[l] += 1
    return A,B

In [311]:
signsH = {}
for i in range(50):
    A,B = count_outperforms(splitted_likes, 'H')
    for i in reversed([2000,1000,250,50]):
            if signsH.has_key(i):
                signsH[i].append(sign_test(A[i],B[i]))
            else:
                signsH[i] = [sign_test(A[i],B[i])]

In [316]:
A,B = count_outperforms(splitted_likes, 'H') 

In [317]:
A

{50: 17, 250: 56, 1000: 88, 2000: 102}

In [318]:
B

{50: 11, 250: 28, 1000: 66, 2000: 76}

In [230]:
from math import factorial as fac
def sign_test(A,B):
    p = 0.
    n = (A+B)
    for i in range(A,n):
        p += fac(n)/ (fac(i)*fac(n-i))
    p *= (0.5)**(n)
    return p

In [296]:
signs = {}
for key in "ABCDEFGHIJ":
    result={}
    for j in range(10):
        A,B = count_outperforms(splitted_likes, key) 
        for i in reversed([2000,1000,250,50]):
            if result.has_key(i):
                result[i].append(sign_test(A[i],B[i]))
            else:
                result[i] = [sign_test(A[i],B[i])]
    for i in reversed([2000,1000,250,50]):
        result[i] = np.mean(result[i])
    signs[key] = result

In [304]:
output = "N & A & B & C & D & E & F & G & H & I & J & \\\\ \\hline \n"
for i in reversed([2000,1000,250,50]):
    output += "$%d$\t&\t" % i
    for key in "ABCDEFGHIJ":
        output += "$%.2f$\t&\t" % signs[key][i]
    output += "\\\\ \\hline \n"

In [305]:
print output

N & A & B & C & D & E & F & G & H & I & J & \\ \hline 
$50$	&	$0.83$	&	$0.32$	&	$0.71$	&	$0.64$	&	$0.25$	&	$0.47$	&	$0.57$	&	$0.07$	&	$0.75$	&	$0.45$	&	\\ \hline 
$250$	&	$0.34$	&	$0.44$	&	$0.57$	&	$0.30$	&	$0.15$	&	$0.54$	&	$0.51$	&	$0.02$	&	$0.38$	&	$0.31$	&	\\ \hline 
$1000$	&	$0.41$	&	$0.18$	&	$0.23$	&	$0.09$	&	$0.08$	&	$0.41$	&	$0.59$	&	$0.08$	&	$0.27$	&	$0.13$	&	\\ \hline 
$2000$	&	$0.06$	&	$0.17$	&	$0.22$	&	$0.11$	&	$0.20$	&	$0.47$	&	$0.49$	&	$0.09$	&	$0.33$	&	$0.04$	&	\\ \hline 



In [359]:
def usage_prediction(like_dict, feature, rec_lengths=[2000,1000,250,50]):
    user_scores = {}
    f = FloatProgress(min=0, max=100)
    display(f)
    for l in rec_lengths:
        user_scores[l] = []
    for i,user in enumerate(like_dict.keys()):
            f.value = float(i)*100/len(like_dict)
            for l in rec_lengths:
                recs = make_recommendations_cluster_scored(feature, like_dict[user]['rec'],2000,data[feature]['data'].T, k=l/50)
                #cf_recs = make_random_recommendations(len(recs)) #make_cf_recommendations(like_dict[user]['rec'], len(recs))
                ev = set(like_dict[user]['eval'])
                rec_set = set(recs[:l])
                #cf_rec_set = set(cf_recs[:l])
                
                tp = len(rec_set & ev) *1.
                fp = len(rec_set - ev) *1.
                fn = len(ev) - tp *1.
                tn = 8003. - (len(rec_set)+fn) * 1.

                #precision, recall, false positive rate
                user_scores[l].append([tp/(tp+fp), tp/(tp+fn), fp/(fp+tn)])
    scores = {}
    for key, values in user_scores.iteritems():
        mean = np.mean(values, axis=0)
        std = np.std(values, axis=0)
        scores[key] = np.asarray(zip(mean,std)).flatten()
    return user_scores,scores

In [406]:
user_scores, scores = usage_prediction(splitted_likes,'B')

In [365]:
for key in sorted(scores.keys()):
    print "$" + str(key) + "$ & $%2.3f\\%% (\\pm %2.3f\\%%)$& $%2.3f\\%% (\\pm %2.3f\\%%)$&$ %2.3f\\%% (\\pm %2.3f\\%%)$ \\\\ \hline" % tuple(scores[key][:6]*100)

$50$ & $0.104\% (\pm 0.479\%)$& $0.874\% (\pm 4.371\%)$&$ 0.625\% (\pm 0.003\%)$ \\ \hline
$250$ & $0.085\% (\pm 0.178\%)$& $3.865\% (\pm 9.363\%)$&$ 3.123\% (\pm 0.005\%)$ \\ \hline
$1000$ & $0.082\% (\pm 0.105\%)$& $13.888\% (\pm 17.772\%)$&$ 12.494\% (\pm 0.011\%)$ \\ \hline
$2000$ & $0.082\% (\pm 0.078\%)$& $27.492\% (\pm 21.363\%)$&$ 24.988\% (\pm 0.013\%)$ \\ \hline


In [117]:
ROC_B = []
for key in sorted(scores.keys()):
    ROC_B.append(scores[key][2])

In [368]:
def rscore(recs, ev, alpha):
    score = 0
    for i,r in enumerate(recs):
        if r in ev:
            score += 1 / 2**(i/alpha - 1)
            #print "hit at %d" % i
    max_score = 0
    for i,r in enumerate(ev):
        max_score += 1 / 2**(i/alpha - 1)
    return score, max_score

In [404]:
def get_rscore(like_dict,alpha):
    rscores = []
    #rscores_b = []
    max_rscore= []
    f = FloatProgress(min=0, max=100)
    display(f)
    for i,user in enumerate(like_dict.keys()):
        f.value = float(i)*100/len(like_dict)
        recs = make_recommendations_cluster_scored('B', like_dict[user]['rec'],100,data['B']['data'].T, k=5)
        #recs = make_random_recommendations(100)
        rscore_user, max_rscore_user = rscore(recs,like_dict[user]['eval'],alpha)
        #print "our %f %f" % (rscore_user, max_rscore_user)
        #rand_rscore_user, max_rscore_user = rscore(rand_recs,like_dict[user]['eval'],alpha)
        #print "rand %f %f \n" % (rand_rscore_user, max_rscore_user)
        rscores.append(rscore_user)
        #rscores_b.append(rand_rscore_user)
        max_rscore.append(max_rscore_user)
    r = 100* sum(rscores)/sum(max_rscore)
    #r_b = 100* sum(rscores_b)/sum(max_rscore)
    return r#,r_b

In [405]:
get_rscore(splitted_likes,0.9)

0.003402817041039896

In [ ]:
0.11685673079686356 #H Rscore

In [ ]:
0.02536004547366449 #A RScore

In [401]:
np.mean(r) #Random Rscore

0.085284235452609936

In [387]:
0.05213995098123266 #D Rscore

0.05213995098123266

In [8]:
new_likes = {}
for key,values in user_likes.iteritems():
    new_likes[key] = Counter(values)

In [10]:
from math import floor
splitted_likes = {}
p = 0.9
for user in new_likes.keys():
    likes = new_likes[user].keys()
    first = int(floor(len(likes) * p))
    last = len(likes) - first
    random.shuffle(likes)
    splitted_likes[user] = {'rec':likes[:first], 'eval':likes[-last:]}

In [11]:
for val in splitted_likes.values():
    if len(set(val['rec']) & set(val['eval'])) > 0:
        print "error"
    if len(val['rec']) < 1 or len(val['eval']) < 1:
        print 'too small'

In [144]:
splitted_likes['eb248402322adb243ca168a9bf624504']

{'eval': ['TROSAAV128F93424E7',
  'TRLWWYP128F92ECD5F',
  'TRGQECM128F4230D0C',
  'TRMVVVK128F9307B40',
  'TRQVUCJ128F92DF5FE'],
 'rec': ['TRSBOOT128F930CD73',
  'TRAATZB12903CC59D8',
  'TRPTHQP128F427261D',
  'TRUINQU128F14A66A6',
  'TRJPEPR12903CD7301',
  'TRDMRMV128F9330FC2',
  'TRZWCRX12903C9700E',
  'TRPMNBR128F92F9588',
  'TRSPVGX128F427AA3D',
  'TREAAMX128F423EAEE',
  'TRTWRDZ128F1452BF8',
  'TRXNRTN12903CC449D',
  'TRDXGYW128F92C5A72',
  'TROLYAE128F934C593',
  'TRBEBDQ128F1469001',
  'TRCBMHE128F149077E',
  'TRANPKC128F93486B9',
  'TRINAJE128F92FB145',
  'TRXZMOQ128F429D983',
  'TRXDECL128F93139E5',
  'TRPUFBX128F93296CB',
  'TROROLJ12903CE6C8B',
  'TRAGNIQ128F92F45F9',
  'TRZIGOU128F425CEC4',
  'TRQXODD128F92D0670',
  'TRDWTEB128F1464BE3',
  'TRUQDBE128F149EC19',
  'TRYQRLB128F93362E2',
  'TREITPK128F4289C6D',
  'TRXVNDR128F145E0FE',
  'TRXNPHC128F147365A',
  'TRPXIBW128F42BA3C0',
  'TRBUYKI128F92DB14A',
  'TRKXNDU128F4263E8C',
  'TRAPLDX128F931C321',
  'TRFLARJ128F930061B']}